<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/06.%20Chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 6: Decision Trees

Bab ini membahas **Decision Trees**, sebuah algoritma yang kuat dan serbaguna yang mampu melakukan tugas klasifikasi, regresi, dan bahkan multioutput. Decision Trees juga merupakan komponen dasar dari **Random Forests**.

* **Pelatihan dan Visualisasi**: Decision Tree adalah model yang intuitif. Model ini belajar dengan membuat serangkaian pertanyaan sederhana tentang data. Hasilnya adalah sebuah struktur seperti pohon di mana setiap *node* internal merepresentasikan sebuah tes pada sebuah fitur, dan setiap *leaf node* (daun) merepresentasikan sebuah hasil (kelas atau nilai). Salah satu kelebihan utamanya adalah hasilnya mudah diinterpretasikan (*white box model*).

* **Membuat Prediksi**: Untuk membuat prediksi, kita mulai dari *root node* (akar) dan menelusuri cabang-cabang pohon berdasarkan hasil tes pada setiap *node* hingga mencapai *leaf node*. Prediksi akhir adalah label atau nilai yang ada di *leaf node* tersebut.

* **Estimasi Probabilitas Kelas**: Selain memprediksi kelas, Decision Tree juga dapat mengestimasi probabilitas sebuah instance termasuk dalam kelas tertentu. Probabilitas ini dihitung sebagai rasio instance dari kelas tersebut di dalam *leaf node* tempat instance tersebut jatuh.

* **Algoritma Training CART**: Scikit-Learn menggunakan algoritma **Classification and Regression Tree (CART)**. Algoritma ini bersifat *greedy*, artinya ia mencari pemisahan (*split*) terbaik di setiap level tanpa mempertimbangkan apakah pemisahan tersebut akan menghasilkan solusi optimal secara global beberapa level di bawahnya.
    * Untuk klasifikasi, CART mencoba meminimalkan **Gini impurity**. Tujuannya adalah menghasilkan *subset* yang paling "murni" (sebagian besar instance di dalamnya berasal dari kelas yang sama).
    * Untuk regresi, CART mencoba meminimalkan **Mean Squared Error (MSE)**.

* **Regularisasi**: Decision Tree sangat rentan terhadap *overfitting* jika tidak dibatasi. Untuk mengaturnya, digunakan *hyperparameter* regularisasi seperti:
    * `max_depth`: Membatasi kedalaman maksimum pohon.
    * `min_samples_split`: Jumlah minimum instance yang harus dimiliki sebuah *node* agar bisa dipecah.
    * `min_samples_leaf`: Jumlah minimum instance yang harus ada di sebuah *leaf node*.

* **Regresi dengan Decision Tree**: Pohon juga bisa digunakan untuk regresi. Prediksi di setiap *leaf node* adalah nilai rata-rata dari target instance yang ada di *node* tersebut. Hasilnya adalah sebuah fungsi prediksi yang berbentuk tangga (*step-wise function*).

* **Kelemahan**: Decision Tree sensitif terhadap variasi kecil dalam data training dan rotasi data. Perubahan kecil pada data dapat menghasilkan pohon yang sangat berbeda.

### 1. Melatih dan Memvisualisasikan Decision Tree untuk Klasifikasi
Kita akan menggunakan dataset Iris yang sudah dikenal.

```python
# Diperlukan untuk visualisasi di Colab
!pip install graphviz

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz

# Memuat dataset Iris
iris = datasets.load_iris()
X = iris.data[:, 2:] # petal length dan width
y = iris.target

# Melatih model DecisionTreeClassifier dengan kedalaman maksimum 2
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

# Mengekspor model ke format .dot
dot_data = export_graphviz(
        tree_clf,
        out_file=None,
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

# Memvisualisasikan pohon
graph = graphviz.Source(dot_data)
graph
```
Output dari sel di atas adalah visualisasi pohon keputusan yang mudah dibaca. Anda bisa melihat kondisi di setiap *node* (misalnya, `petal width (cm) <= 0.8`), nilai Gini, jumlah *samples*, dan kelas prediksi.

### 2. Membuat Prediksi dan Estimasi Probabilitas
Setelah pohon dilatih, kita bisa menggunakannya untuk prediksi.

```python
# Misalkan kita punya bunga dengan petal length 5cm dan petal width 1.5cm
# Prediksi probabilitas kelas
probabilities = tree_clf.predict_proba([[5, 1.5]])
print("Estimasi Probabilitas:", probabilities)

# Prediksi kelas
predicted_class = tree_clf.predict([[5, 1.5]])
print("Prediksi Kelas:", predicted_class)
print(f"Nama Kelas: {iris.target_names[predicted_class[0]]}")
```
Berdasarkan pohon yang divisualisasikan, instance ini akan jatuh ke *node* tengah (`value = [0, 49, 5]`), sehingga probabilitas tertinggi adalah untuk kelas *Iris-Versicolor*.

### 3. Melatih Decision Tree untuk Regresi
Decision Tree juga bisa digunakan untuk tugas regresi.

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

# Membuat data kuadratik dengan noise
np.random.seed(42)
m = 200
X = np.random.rand(m, 1) * 10 - 5
y = 0.5 * X**2 + 2 * X + 1 + np.random.randn(m, 1)

# Melatih model DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[-5, 5, -2, 10]):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$")
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

# Plot hasil prediksi
plt.figure(figsize=(8, 4))
plot_regression_predictions(tree_reg, X, y)
plt.title("Prediksi Regresi dengan Decision Tree (max_depth=2)")
plt.legend()
plt.show()
```
Plot di atas menunjukkan bahwa prediksi model regresi Decision Tree adalah nilai konstan di dalam setiap wilayah (daun), menghasilkan bentuk seperti tangga. Ini adalah karakteristik utama dari regresi dengan Decision Tree.

